In [1]:
from random import shuffle
from tqdm import tqdm
import re
import os
import sys
sys.path.append('../rusenteval')

In [2]:
import stanza
ppln = stanza.Pipeline('ru', processors='tokenize,pos')

2023-11-19 01:50:57 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-11-19 01:50:58 INFO: Loading these models for language: ru (Russian):
| Processor | Package          |
--------------------------------
| tokenize  | syntagrus        |
| pos       | syntagrus_charlm |

2023-11-19 01:50:58 INFO: Using device: cpu
2023-11-19 01:50:58 INFO: Loading: tokenize
2023-11-19 01:50:58 INFO: Loading: pos
2023-11-19 01:50:59 INFO: Done loading processors!


In [3]:
numbers = []
def check_sents(text):
        text = re.sub(r'[^\w\s]','',text)[:-1]
        labels = []
        doc = ppln(text)
        for i in range(len(doc.sentences[0].words)):
            if doc.sentences[0].words[i].upos == 'ADJ':
                feats = doc.sentences[0].words[i].feats.split('|')
                data_dict = {item.split('=')[0]: item.split('=')[1] for item in feats}
                if len(data_dict) > 2:
                    if data_dict['Number'] == 'Plur' or data_dict['Number'] == 'Sing':
                        labels.append(data_dict['Number'])
                        numbers.append(data_dict['Number'])
                    else:
                        labels.append('None')     
                else:
                    labels.append('None')  
            else:
                labels.append('None')
        return text, labels   

In [4]:
with open('../rusenteval/obj_number.txt') as f:
    sents = [line.strip('\n').split('\t') for line in f]
shuffle(sents)
sents = [d[2] for d in sents if 4 < len(d[2].split()) <= 20][:40000]

In [5]:
new_sents = []
for sent in tqdm(sents):
    new_sents.append(check_sents(sent))

100%|███████████████████████████████████| 40000/40000 [5:31:30<00:00,  2.01it/s]


In [6]:
new_sents = [d for d in new_sents if d is not None]
new_sents = [d for d in new_sents if d[1].count('None') != len(d[1]) and len(d[0].split()) == len(d[1])]

In [7]:
per_token = [d for d in new_sents if d[1].count('None') <= len(d[1]) - 2 and len(d[0].split()) == len(d[1])]

In [8]:
per_sent = [d for d in new_sents if d[1].count('None') + 1 == len(d[1]) and len(d[0].split()) == len(d[1])]

In [9]:
len(per_token)

11339

In [10]:
len(per_sent)

14716

In [11]:
set(numbers)

{'Plur', 'Sing'}

## per_token

In [144]:
shuffle(per_token)

In [145]:
sg = []
pl = []
for text in per_token:
    if text[1].count('Sing') > 0:
        sg.append(text)
    elif text[1].count('Plur') > 0:
        pl.append(text)

In [146]:
from random import shuffle
shuffle(sg)
shuffle(pl)
sg = sg[:1950]
pl = pl[:1050]

In [147]:
len(sg)

1950

In [148]:
sg.extend(pl)

In [149]:
len(sg)

3000

In [150]:
shuffle(sg)

In [151]:
filename_out = '/per_token/ru_rusenteval_ADJ_Number.txt'
with open(filename_out, 'w', encoding='UTF-8') as output:
    for line in sg:
        output.write(line[0])
        output.write('\n')
        output.write(str(line[1])[1:-1])
        output.write('\n')

In [152]:
sg_count = 0
pl_count = 0
for text in sg:
    sg_count += text[1].count('Sing')
    pl_count += text[1].count('Plur')
print(sg_count, pl_count)

3746 3745


## per_sent

In [20]:
sg = []
pl = []
for text in per_sent:
    if text[1].count('Sing') > 0:
        sg.append(text)
    elif text[1].count('Plur') > 0:
        pl.append(text)

In [21]:
from random import shuffle
shuffle(sg)
shuffle(pl)
sg = sg[:1500]
pl = pl[:1500]

In [22]:
len(sg)

1500

In [23]:
sg.extend(pl)

In [24]:
len(sg)

3000

In [30]:
shuffle(sg)

In [34]:
# сохраним в файл
filename_out = '/per_sent/ru_rusenteval_ADJ_Number.txt'
with open(filename_out, 'w', encoding='UTF-8') as output:
    for line in sg:
        output.write(line[0])
        output.write('\n')
        label = [label for label in line[1] if label != 'None']
        assert len(label) == 1
        output.write(label[0])
        output.write('\n')

In [35]:
sg_count = 0
pl_count = 0
for text in sg:
    sg_count += text[1].count('Sing')
    pl_count += text[1].count('Plur')
print(sg_count, pl_count)

1500 1500
